In [4]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [30]:
df = (
    pl.scan_csv("../data/processed/df_procesada.csv", separator=";")
    .filter((pl.col("COD_HOSPITAL") == 112103) & (pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN"))
    .group_by(["ANIO_EGRESO", "DIAGNOSTICO1"])
    .agg(
        [
            pl.col("CIP_ENCRIPTADO").n_unique().alias("n_pacientes_distintos"),
            pl.len().alias("n_egresos"),
            pl.col("IR_29301_PESO").mean().round(4).alias("peso_grd_promedio"),
            pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
        ]
    )
    .sort(by=["ANIO_EGRESO", "n_egresos"], descending=True)
)

In [31]:
df = df.collect(streaming=True)

In [32]:
df

ANIO_EGRESO,DIAGNOSTICO1,n_pacientes_distintos,n_egresos,peso_grd_promedio,dias_estada_totales
i64,str,u32,u32,f64,i64
2021,"""U07.1""",448,449,4.0172,8768
2021,"""I25.1""",285,304,2.5258,2784
2021,"""I35.0""",114,122,3.7032,1404
2021,"""Z45.0""",107,109,2.2902,124
2021,"""C34.1""",60,74,1.5312,550
…,…,…,…,…,…
2019,"""I48.2""",1,1,1.8191,1
2019,"""Q89.3""",1,1,0.6736,2
2019,"""Q24.8""",1,1,1.0746,1
